<a href="https://colab.research.google.com/github/s-neela/neela_INFO5731_spring2021/blob/main/Copy_of_In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (20 points in total, 2/9/2021)**

# 1. Text Data Preprocessing

Here is a [legal case](https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



## 1.1 Basic feature extraction using text data (4 points)

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

## 1.2 Basic Text Pre-processing of text data (4 points)

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

## 1.3 Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above. (4 points)


## 1.4 Advance Text Processing (Extra credit: 4 points)

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [ ]:
# Write your code here

# download the case file
import requests
def download_legalCaseDoc(url, file_name):
  response = requests.get(url)
  with open(file_name, 'wb') as fileWriter:
    fileWriter.write(response.content)
file_name = "legal_case.txt"
download_legalCaseDoc("https://raw.githubusercontent.com/unt-iialab/info5731_spring2021/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt", file_name)


# 1.1 Basic feature extraction using text data'

import numpy as np 
import nltk
nltk.download('stopwords')

def extract_file_content(legalCaseFile):
  with open(legalCaseFile) as legalContentFile:
    legalContent = legalContentFile.read()
    return legalContent.replace('\n','')

def extract_sentences(legalContent):
  rawlines = { line for line in legalContent.split('.')}
  linesWithSentences = list(filter(None, rawlines))
  return linesWithSentences

def extract_charecters(word):
  return list(word)

def find_stopWords(words):
  for word in words:
    if word in en_stops:
      yield word

legalContent = extract_file_content(file_name)
sentences = extract_sentences(legalContent)
npSentences = np.array(sentences)
npWords = np.concatenate(np.char.split(npSentences), axis=0) # Spread array 
stopWords = list(find_stopWords(npWords))
npCharecters = np.concatenate([ extract_charecters(word) for word in npWords], axis=0)
averageLengthOfWords = sum(len(word) for word in npWords)/len(npWords)
special_charecters = find_special_charecters(legalContent)
print('\n# 1.1 Basic feature extraction using text data\n')
print(f'Number of Sentences : {len(npSentences)}')
print(f'Nunmber of Words: {len(npWords)}')
print(f'Number of Charecters: {len(npCharecters)}')
print(f'Average Length Of Word: {averageLengthOfWords} , rounded to : {round(averageLengthOfWords)}')
print(f'Number of stopwords: {len(stopWords)}') 
print(f'Number of special characters: {len(find_special_charecters(legalContent))}')
print(f'Number of numerics: {len(find_numbers(legalContent))}')
print(f'Number of uppercase words: {len(list(find_uppdercaseWords(npWords)))}')


# 1.2 Basic Text Pre-processing of text data
import string
import pandas as pd
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

def print_data(dataFrame):
  print(dataFrame['Sentence'].head())

tokenized_sentences = nltk.tokenize.sent_tokenize(legalContent)
dataFrame = pd.DataFrame(tokenized_sentences, columns=['Sentence'])

print('\nLower casing\n')
dataFrame['Sentence']= dataFrame['Sentence'].apply(lambda sentence: sentence.lower())
print_data(dataFrame)

print('\nPunctuation removal\n')
dataFrame['Sentence']= dataFrame['Sentence'].apply(lambda sentence: sentence.translate(str.maketrans('', '', string.punctuation)))
print_data(dataFrame)

print('\nStopwords removal\n')
freq=pd.Series(' '.join(dataFrame['Sentence']).split()).value_counts()[:10]
dataFrame['Sentence'] = dataFrame['Sentence'].apply(lambda sentence: " ".join(word for word in sentence.split() if word not in stopWords))
print_data(dataFrame)

print('\nFrequent words removal\n')
freq_words=pd.Series(' '.join(dataFrame['Sentence']).split()).value_counts()[:10].index
dataFrame['Sentence'] = dataFrame['Sentence'].apply(lambda sentence: " ".join(word for word in sentence.split() if word not in freq_words))
print_data(dataFrame)

print('\nRare words removal\n')
rare_words=pd.Series(' '.join(dataFrame['Sentence']).split()).value_counts()[-10:].index
dataFrame['Sentence'] = dataFrame['Sentence'].apply(lambda sentence: " ".join(word for word in sentence.split() if word not in rare_words))
print_data(dataFrame)

print('\nSpelling correction\n')
from textblob import TextBlob
dataFrame['Sentence'] = dataFrame['Sentence'].apply(lambda sentence: " ".join([str(TextBlob(word).correct()) for word in sentence.split()]))
print_data(dataFrame)

print('\nTokenization\n')
print(TextBlob(dataFrame['Sentence'][1]).words)

print('\nStemming\n')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
print(dataFrame['Sentence'].apply(lambda sentence: " ".join([stemmer.stem(word) for word in sentence.split()])))

print('\Lemmatization\n')
from textblob import Word
nltk.download('wordnet')
dataFrame['Sentence'] = dataFrame['Sentence'].apply(lambda sentence: " ".join([Word(word).lemmatize() for word in sentence.split()]))
print_data(dataFrame)


#1.3 Save all the clean sentences to a csv file
processed_file_name = 'processed_data.csv'
print(f'\nSaved into file : {processed_file_name}\n')
dataFrame.to_csv(processed_file_name, index=False)

#1.4 Advance Text Processing
print('\nCalculate the term frequency of all the terms.\n')
tf = (dataFrame['Sentence'][1:2]).apply(lambda sentence:pd.value_counts(sentence.split())).sum(axis=0).reset_index()
tf.columns=['Term','frequency']
print(tf)

print('\nPrint out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.\n')
print(TextBlob(dataFrame['Sentence'][10]).ngrams(1))
print(TextBlob(dataFrame['Sentence'][10]).ngrams(2))
print(TextBlob(dataFrame['Sentence'][10]).ngrams(3))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

# 1.1 Basic feature extraction using text data

Number of Sentences : 255
Nunmber of Words: 3599
Number of Charecters: 16286
Average Length Of Word: 4.525145873853848 , rounded to : 5
Number of stopwords: 1670
Number of special characters: 818
Number of numerics: 156
Number of uppercase words: 419
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

Lower casing

0    5 ala. 740supreme court of alabama.adamsv.tann...
1    west headnotes (2)[1]chattel mortgagescropsa g...
2                                                  fa.
3    on a growing crop, nor does such lien attach u...
4    in november, 1840, an execution issued from th...
Name: Sentence, dtype: object

Punctuation removal

0    5 ala 740supreme court of alabamaadamsvtanner ...
1    west headnotes 21chattel mortgagescropsa growi...
2                 

# 2. Python Regular Expression

## 2.1 Write a Python program to remove leading zeros from an IP address. (4 points)

ip = "260.08.094.109"

In [1]:
# Write your code here
import re

def remove_leadingZeros(ip):
  result = re.sub(r'\b0+(\d)', r'\1', ip)
  return result

ip = "260.08.094.109"

print(remove_leadingZeros(ip))

260.8.94.109


## 2.2 Write a Python Program to extract all the years from the following sentence. (4 points)

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [2]:
# Write your code here
import re

def valid_year(year):
  if year and year.isdigit():
    if int(year) >=1900 and int(year) <=2020:
      return int(year)  #return an integer

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
all_4digitNumbers = re.findall(r'\d{4}', sentence)
years = [number for number in all_4digitNumbers if valid_year(number)]
print(years)


['2010', '2010', '2019']
